In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv") #df dataset okutuldu.

In [ ]:
df.describe()  #veri seti tanıtıldı.

In [ ]:
print(df.groupby('Province/State').size()) #ülkeler sınıfına göre verinin bilgisi ve uzunluğunun öğrenilmesi.

In [ ]:
df.head(30)       #datanın ilk 30 satırını görüntüledik

In [ ]:
df.tail(30)       #son 30 satırını görüntüledik.Verinin doğruluna bakıldı.

In [ ]:
df = df.rename(columns={'Province/State': 'Province','Country/Region':'Country','ObservationDate':'Date','Confirmed':'Cases'})

#data setindeki sütun isimleri kısaltıldı.

In [ ]:
df.describe().T

In [ ]:
df.Country.replace({'Mainland China': 'China'}, inplace=True)
df['Date'] = df['Date'].apply(pd.to_datetime)                         #Verideki covid-19 un başladığı ve yayıldığı yer anayer_cin cin olarak değiştirildi.
df.drop(['SNo'],axis=1,inplace=True)
df.drop(list(df[df.Cases==0.0].index), axis = 0, inplace=True)

In [ ]:
df.isnull().sum()                   #eksik veriler icin 

In [ ]:
dfmissing=df[df['Province'].isnull()]['Country'].value_counts().to_frame(name='Missing_Province_count')
dfcountry=df['Country'].value_counts().to_frame(name='Country_count')
mergedDf = dfcountry.merge(dfmissing, left_index=True, right_index=True)

In [ ]:
mergedDf[mergedDf.Missing_Province_count>1]

In [ ]:
df[df.Country=='Australia']=df[df.Country=='Australia'].fillna(df[df.Country=='Australia']["Province"].mode()[0])

df["Province"]=df["Province"].fillna(df["Country"]) #Austrulia verilerinde gerçekten yararlı bir bilgi olmadığı için ülke adları atandı.

In [ ]:
df.sort_values(by='Cases', ascending=False)['Country'].unique()
                                            #vaka sayısına gore azalan ulkeleri sıralama islemi

In [ ]:
df[df.Country=='Others']

In [ ]:
df.Country.replace({'Others': 'Japan'}, inplace=True)

In [ ]:
countries=df.sort_values(by='Cases', ascending=False)['Country'].unique()

In [ ]:
print("\nVirusten etkilenen ulke sayisi: ",len(countries))

In [ ]:
df.groupby("Country").aggregate(['mean', np.std,max,'count']).sort_values([('Cases','max')], ascending=False).head(40)
                                # ilk 40 ülke için yüksekten düşüğe doğru vaka sayıları sıralaması.

In [ ]:
df.groupby("Country").aggregate(['mean', np.std,max,'count']).sort_values([('Deaths','max')], ascending=False).head(40)
                                # ilk 40 ülke için yüksekten düşüğe doğru ölüm sayıları sıralaması.

In [ ]:
dftotal=df.groupby(['Date','Country'])['Cases','Deaths','Recovered'].sum()
dftotal.reset_index(inplace=True)  
dftotal['Death_rate']=dftotal['Deaths']/dftotal['Cases']
dftotal['Recovery_rate']=dftotal['Recovered']/dftotal['Cases']

In [ ]:
dftotal[dftotal.Country=='US']

In [ ]:
 dftotal['log(Cases)']=np.log(dftotal.Cases)  # En yüksek sayılara göre ilk 10 ülke

In [ ]:
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
sns.set_style("whitegrid")
sns.lineplot(x="Date", y="log(Cases)", hue='Country',linewidth=6,  data=dftotal[dftotal.Country.isin(countries[0:10])]);
plt.xticks(rotation=45,ha='right');                                      #en yüksek vaka sayılarına gore ulkeler
sns.set(rc={'figure.figsize':(14,14)})            
leg = plt.legend(fontsize='x-large',loc=2, facecolor='white', )
for i in leg.legendHandles:
    i.set_linewidth(10.0)                        

In [ ]:
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
sns.set_style("whitegrid")
sns.lineplot(x="Date", y="log(Cases)", hue='Country',linewidth=6,  data=dftotal[dftotal.Country.isin(countries[10:30])]);
plt.xticks(rotation=45,ha='right');
sns.set(rc={'figure.figsize':(14,14)})                                    
leg = plt.legend(fontsize='x-large',loc=2, facecolor='white', )
for i in leg.legendHandles:
    i.set_linewidth(10.0)

In [ ]:
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
sns.set_style("whitegrid")
sns.lineplot(x="Date", y="Death_rate", hue='Country',linewidth=6,  data=dftotal[(dftotal.Country.isin(countries[10:29]))&(dftotal.Date>'2020-03-15')]);
plt.xticks(rotation=45,ha='right');
sns.set(rc={'figure.figsize':(14,14)})
leg = plt.legend(fontsize='x-large',loc=2, facecolor='white', )               #15/03/2020 sonrası vakalar

for i in leg.legendHandles:
    i.set_linewidth(10.0)

In [ ]:
country = df.groupby('Country').sum().apply(list).reset_index()
country

In [ ]:
country['%Recovered'] = ((country['Recovered'] / country['Cases'] ) * 100)
country['%Recovered'] = country['%Recovered'].round(2)
country

In [ ]:
from datetime import datetime,date

df['date'] = pd.to_datetime(df['Last Update']).dt.date
df

In [ ]:
date_conf_cases = df.groupby('date').sum().apply(list).reset_index()
date_conf_cases

In [ ]:
date_conf_cases['Cases'].sum()

In [ ]:
i=1
tot_conf = 0
date_conf_cases['Total Confirmed Cases'] = 1
date_conf_cases['Days'] = 1
for ind in date_conf_cases.index: 
    date_conf_cases['Days'][ind] = i
    i=i+1
    date_conf_cases['Total Confirmed Cases'][ind] = date_conf_cases['Cases'][ind] + tot_conf
    tot_conf = date_conf_cases['Total Confirmed Cases'][ind]
date_conf_cases

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()

In [ ]:
x= date_conf_cases['Days']
y=date_conf_cases['Total Confirmed Cases']
x_matrix=x.values.reshape(-1,1)

In [ ]:
reg.fit(x_matrix,y)

In [ ]:
reg.intercept_

In [ ]:
reg.score(x_matrix,y)

In [ ]:
new_data=pd.DataFrame(data=[3650,7300,10950],columns=['Gunler'])
new_data

#regresion model yaratıldı. Salgının onumuzdeki gunlerde nasıl hareket edecegi ve nasıl artıs gosterecegını kontrol edilecek.

#3 farklı zaman datası denendi. 10 yıl(3650 gun) -  20 yıl (7300 gun)- 30 yıl (10950 gun )

In [ ]:
new_data_matrix=new_data.values.reshape(-1,1)                              
reg.predict(new_data_matrix)

In [ ]:
# Etkilenen Alanın çoğu yogun nufuslu ulkeler. Bu ulkelerın basında cin ve hindistan geliyor.
# Yaptıgımız cıkarımlara baktıgımızda su sonucları elde ederiz.
#10 yıl icerisinde 21 milyon kisi etkilenecek
#20  yıl icerisinde 43 milyon kisi ve
#30 yıl icerisinde 64 milyon kisi.